## LightGBM

Abrimos el dataset en el cual ya hemos trabajado las nuevas variables y caracteristicas.<br><br>
NOTAS:<br>
- algunas no fraude eliminar
- generar
- eliminar y aumentar
- matriz de confusión (clase objetivo) (clase 2)

In [44]:
#!pip install lightgbm
#!pip install --upgrade lightgbm
#!pip install category_encoders
#!pip install geopy

In [45]:
import pandas as pd
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
import numpy as np
import logging
import os
import shutil
import seaborn as sns
import matplotlib.pyplot as plt

In [46]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

In [47]:
data = pd.read_csv('final_dataset.csv')
data

,trans_date_trans_time,cc_num,merchant,category,amt,lat,long,trans_num,unix_time,merch_lat,...,yearly_txn_count_by_cat,season,seasonal_txn_count_by_cat,time_diff_hours,transaction_speed,amt_to_daily_avg,amt_to_monthly_avg,txn_count_last_hour,new_merchant,amt_to_annual_avg
0,2019-01-03 01:39:04,180011453250192,"fraud_Huel, Hammes and Witting",grocery_pos,90.37,41.2153,-90.9879,96a9fd4073d93d59eca9622dc21802c4,1325554744,41.810204,...,1,Winter,1,NaN,0.000000,0.225391,0.804453,1.0,True,0.925752
1,2019-01-03 05:51:22,180011453250192,fraud_Huel-Langworth,misc_net,108.09,41.2153,-90.9879,04cf40b2f392f3932b2532a92f1ef538,1325569882,41.016419,...,1,Winter,1,4.0,11.951459,0.269586,0.962193,1.0,True,1.107276
2,2019-01-03 14:48:47,180011453250192,fraud_Lemke and Sons,travel,447.52,41.2153,-90.9879,90d74d1a15d29a250fedbabe6721a483,1325602127,41.734914,...,1,Winter,1,8.0,8.712667,1.116156,3.983721,1.0,True,4.584404
3,2019-01-04 10:39:10,180011453250192,"fraud_Christiansen, Goyette and Schamberger",gas_transport,73.17,41.2153,-90.9879,52f1b4392482195579c35df07514962a,1325673550,40.744024,...,1,Winter,1,19.0,3.096099,0.182493,0.651343,1.0,True,0.749555
4,2019-01-04 12:48:21,180011453250192,fraud_Ruecker-Mayert,kids_pets,47.55,41.2153,-90.9879,63954f859ede83d209aece226970441b,1325681301,41.462258,...,1,Winter,1,2.0,41.738353,0.118594,0.423279,1.0,True,0.487103
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1852389,2020-12-31 14:49:51,676372984911,fraud_Kris-Padberg,shopping_pos,6.13,39.7562,-98.4462,e327aa916b4317b8baf9d181855bf9df,1388501391,39.553152,...,145,Winter,96,3.0,24.693907,0.019995,0.087796,1.0,False,0.094527
1852390,2020-12-31 16:17:19,676372984911,fraud_Schiller Ltd,personal_care,62.99,39.7562,-98.4462,2d10acccfe94d134bdac9c41c614cb00,1388506639,40.024226,...,145,Winter,95,1.0,34.733267,0.205466,0.902169,1.0,False,0.971327
1852391,2020-12-31 20:21:23,676372984911,fraud_Pollich LLC,home,28.11,39.7562,-98.4462,24ffee6f547efdbffcf471498e55e0eb,1388521283,40.412189,...,195,Winter,88,4.0,21.983892,0.091692,0.402603,1.0,False,0.433466
1852392,2020-12-31 20:58:25,676372984911,"fraud_Hayes, Marquardt and Dibbert",misc_net,211.32,39.7562,-98.4462,898279ea0067e89917bdf288f96d6834,1388523505,40.240147,...,91,Winter,36,0.0,inf,0.689301,3.026613,2.0,False,3.258625


In [48]:
data.columns

Index(['trans_date_trans_time', 'cc_num', 'merchant', 'category', 'amt', 'lat',
       'long', 'trans_num', 'unix_time', 'merch_lat', 'merch_long', 'is_fraud',
       'amt_month', 'amt_year', 'amt_month_shopping_net_spend',
       'count_month_shopping_net', 'first_time_at_merchant',
       'time_since_last_txn', 'avg_daily_amt', 'hour',
       'avg_hourly_transactions', 'prev_lat', 'prev_long',
       'distance_from_last_txn', 'month_year', 'monthly_spending_variation',
       'weekday', 'is_holiday', 'first_txn_time', 'days_since_first_txn',
       'txn_count_last_24h', 'avg_amt_last_24h', 'std_amt_last_24h',
       'online_txn_ratio', 'trans_date', 'monthly_txn_count_by_cat',
       'trans_year', 'yearly_txn_count_by_cat', 'season',
       'seasonal_txn_count_by_cat', 'time_diff_hours', 'transaction_speed',
       'amt_to_daily_avg', 'amt_to_monthly_avg', 'txn_count_last_hour',
       'new_merchant', 'amt_to_annual_avg'],
      dtype='object')

Vamos a convertir aquellas columnas que no sean datos numericos con un encoder

In [49]:
columns_to_encode = ['merchant', 'category', 'trans_num', 'cc_num']

with open('label_encodings.txt', 'w') as file:
    for col in columns_to_encode:
        encoder = LabelEncoder()
        data[col + '_encoded'] = encoder.fit_transform(data[col])
        mappings = {index: label for index, label in enumerate(encoder.classes_)}
        file.write(f"{col} mappings:\n")
        for key, value in mappings.items():
            file.write(f"{key}: {value}\n")
        file.write("\n")

data.drop(columns=columns_to_encode, axis=1, inplace=True)

In [50]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1852394 entries, 0 to 1852393
Data columns (total 47 columns):
 #   Column                        Dtype  
---  ------                        -----  
 0   trans_date_trans_time         object 
 1   amt                           float64
 2   lat                           float64
 3   long                          float64
 4   unix_time                     int64  
 5   merch_lat                     float64
 6   merch_long                    float64
 7   is_fraud                      int64  
 8   amt_month                     float64
 9   amt_year                      float64
 10  amt_month_shopping_net_spend  float64
 11  count_month_shopping_net      float64
 12  first_time_at_merchant        bool   
 13  time_since_last_txn           float64
 14  avg_daily_amt                 float64
 15  hour                          int64  
 16  avg_hourly_transactions       float64
 17  prev_lat                      float64
 18  prev_long             

Observamos detalladamente por aquellas columnas que no son numéricas que pueden necesitas codificación:

In [51]:
non_numeric_columns = data.select_dtypes(include=['object']).columns

print("Columnas no numéricas que podrían necesitar codificación:")
non_numeric_columns

Columnas no numéricas que podrían necesitar codificación:


Index(['trans_date_trans_time', 'month_year', 'first_txn_time', 'trans_date',
       'season'],
      dtype='object')

Ahora vamos a generar los sub datasets donde la información de cada mes será agrupada en un solo set para poder hacer entrenamiento incremental por mes.

In [52]:
data['trans_date_trans_time'] = pd.to_datetime(data['trans_date_trans_time'])
data.set_index('trans_date_trans_time', inplace=True)

In [53]:
monthly_groups = data.groupby(pd.Grouper(freq='M'))
monthly_datasets = {}
for name, group in monthly_groups:
    print(f"Datos para el mes: {name}")
    monthly_datasets[name] = group
    print(group.head())
    print("\n-------------------------------------------\n")


Datos para el mes: 2019-01-31 00:00:00
                          amt      lat      long   unix_time  merch_lat  \
trans_date_trans_time                                                     
2019-01-01 00:00:18      4.97  36.0788  -81.1781  1325376018  36.011293   
2019-01-01 00:00:44    107.23  48.8878 -118.2105  1325376044  49.159047   
2019-01-01 00:00:51    220.11  42.1808 -112.2620  1325376051  43.150704   
2019-01-01 00:01:16     45.00  46.2306 -112.1138  1325376076  47.034331   
2019-01-01 00:03:06     41.96  38.4207  -79.4629  1325376186  38.674999   

                       merch_long  is_fraud  amt_month  amt_year  \
trans_date_trans_time                                              
2019-01-01 00:00:18    -82.048315         0       4.97      4.97   
2019-01-01 00:00:44   -118.186462         0     107.23    107.23   
2019-01-01 00:00:51   -112.154481         0     220.11    220.11   
2019-01-01 00:01:16   -112.561071         0      45.00     45.00   
2019-01-01 00:03:06    -78.

Procedemos a separar el dataset en 80% entrenamiento y 20% validación.

In [54]:
data = pd.read_csv('fraud_feature_engineering_example.csv')
X = data.drop('is_fraud', axis=1)
y = data['is_fraud']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

In [55]:
X_train.isnull().sum()

trans_date_trans_time           0
cc_num                          0
merchant                        0
category                        0
amt                             0
first                           0
last                            0
gender                          0
street                          0
city                            0
state                           0
zip                             0
lat                             0
long                            0
city_pop                        0
job                             0
dob                             0
trans_num                       0
unix_time                       0
merch_lat                       0
merch_long                      0
amt_month                       0
amt_year                        0
amt_month_shopping_net_spend    0
count_month_shopping_net        0
first_time_at_merchant          0
dtype: int64

In [56]:
X_train.describe()

,cc_num,amt,zip,lat,long,city_pop,unix_time,merch_lat,merch_long,amt_month,amt_year,amt_month_shopping_net_spend,count_month_shopping_net
count,1.111436e+06,1.111436e+06,1.111436e+06,1.111436e+06,1.111436e+06,1.111436e+06,1.111436e+06,1.111436e+06,1.111436e+06,1.111436e+06,1.111436e+06,1.111436e+06,1.111436e+06
mean,4.179349e+17,6.992089e+01,4.877639e+04,3.853877e+01,-9.020667e+01,8.840666e+04,1.358668e+09,3.853879e+01,-9.020631e+01,4.156127e+03,4.531528e+04,3.761065e+02,4.571047e+00
std,1.309950e+18,1.540079e+02,2.688275e+04,5.063971e+00,1.373079e+01,3.010838e+05,1.819268e+07,5.098405e+00,1.374231e+01,3.908474e+03,3.587280e+04,7.245533e+02,4.577667e+00
min,6.041621e+10,1.000000e+00,1.257000e+03,2.002710e+01,-1.656723e+02,2.300000e+01,1.325376e+09,1.902742e+01,-1.666707e+02,1.000000e+00,1.030000e+00,0.000000e+00,0.000000e+00
25%,1.800400e+14,9.640000e+00,2.623700e+04,3.466890e+01,-9.679800e+01,7.430000e+02,1.343014e+09,3.474374e+01,-9.688485e+01,1.346807e+03,1.734013e+04,9.020000e+00,1.000000e+00
50%,3.521417e+15,4.745000e+01,4.817400e+04,3.935430e+01,-8.746160e+01,2.443000e+03,1.357071e+09,3.936946e+01,-8.741253e+01,3.074275e+03,3.743163e+04,7.595000e+01,3.000000e+00
75%,4.642255e+15,8.308000e+01,7.201100e+04,4.194040e+01,-8.015800e+01,2.032800e+04,1.374575e+09,4.195490e+01,-8.023103e+01,5.741590e+03,6.473645e+04,4.262400e+02,7.000000e+00
max,4.992346e+18,2.894890e+04,9.992100e+04,6.669330e+01,-6.795030e+01,2.906700e+06,1.388534e+09,6.744152e+01,-6.695203e+01,4.326189e+04,2.190868e+05,1.204718e+04,4.800000e+01


### Entrenamiento con LGBM

In [57]:
params = {
    'objective': 'binary',
    'metric': 'binary_logloss',
    'verbose': -1
}
lgb_model = None

# Iterar sobre cada segmento mensual
for name, month_data in monthly_groups:
    print(f"Entrenando con datos de: {name}")

    X_train_month = month_data.drop(['is_fraud'], axis=1)
    y_train_month = month_data['is_fraud']
    train_data_month = lgb.Dataset(X_train_month, label=y_train_month)

    if lgb_model is None:
        lgb_model = lgb.train(params, train_data_month, num_boost_round=100)
    else:
        # Entrenamiento incremental
        lgb_model = lgb.train(params, train_data_month, init_model=lgb_model, num_boost_round=10)

Entrenando con datos de: 2019-01-31 00:00:00


ValueError: pandas dtypes must be int, float or bool.
Fields with bad pandas dtypes: month_year: object, first_txn_time: object, trans_date: object, season: object

In [ ]:
y_pred = (lgb_model.predict(X_test) > 0.5).astype(int)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')